In [1]:
#%cd C:\Users\itaye\Desktop\pdexplain\pd-explain\src
%cd C:\Users\User\Desktop\pd_explain_test\pd-explain\src

C:\Users\User\Desktop\pd_explain_test\pd-explain\src


In [2]:
import pandas as pd
import pd_explain

C:\Users\User\AppData\Local\Temp\ipykernel_17600\2046952530.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


usetex-False


In [3]:
#spotify_all = (pd.read_csv(r"C:\Users\itaye\Desktop\pdexplain\pd-explain\Examples\Datasets\spotify_all.csv"))
spotify_all = (pd.read_csv(r"C:\Users\User\Desktop\pd_explain_test\pd-explain\Examples\Datasets\spotify_all.csv"))

In [4]:
spotify_all['main_artist'] = spotify_all['main_artist'].replace({'\$': ''}, regex=True)

<>:1: SyntaxWarning: invalid escape sequence '\$'
<>:1: SyntaxWarning: invalid escape sequence '\$'
C:\Users\User\AppData\Local\Temp\ipykernel_19664\3448523466.py:1: SyntaxWarning: invalid escape sequence '\$'
  spotify_all['main_artist'] = spotify_all['main_artist'].replace({'\$': ''}, regex=True)


In [4]:
grouped3 = spotify_all.groupby(['main_artist'])['energy'].mean()

grouped3.explain()

Error in callback <function _draw_all_if_interactive at 0x0000015D850AF100> (for post_execute), with arguments args (),kwargs {}:


ValueError: 
\bf{'main artist' = '
                     ^
ParseFatalException: Expected '}', found end of text  (at char 21), (line:1, col:22)

ValueError: 
\bf{'main artist' = '
                     ^
ParseFatalException: Expected '}', found end of text  (at char 21), (line:1, col:22)

<Figure size 700x700 with 1 Axes>